<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install --upgrade --force-reinstall nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled

In [5]:
import torch
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup

# Ensure NLTK is correctly installed and restart the kernel
nltk.download('wordnet')

pd.set_option('display.max_colwidth', None)

AttributeError: module 'nltk' has no attribute 'internals'

In [ ]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [ ]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)

In [ ]:
df

In [ ]:
balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
balanced_df

In [ ]:
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

balanced_df['sentiment'] = balanced_df['star_rating'].apply(map_sentiment)
orig_df = balanced_df.copy()
balanced_df

In [ ]:
#train_df, test_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['sentiment'], random_state=42)
print(orig_df)

In [ ]:
pip install gensim

In [ ]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# Download NLTK tokenizer
nltk.download('punkt')
import gensim.downloader as api
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# Download tokenizer
nltk.download('punkt')


In [ ]:
# Load pretrained Word2Vec model (Google News 300)
wv = api.load("word2vec-google-news-300")

# Check the vector representation of a word
#print(wv["excellent"])  # A 300-dimensional vector

In [ ]:
#check semantic similarities
result = wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("King - Man + Woman =", result[0][0])  # Expected output: 'queen'

In [ ]:
#similarities between words
similarity = wv.similarity("excellent", "outstanding")
print("Similarity between 'excellent' and 'outstanding':", similarity)

In [ ]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
balanced_df["tokens"] = balanced_df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(balanced_df["tokens"].head())


In [ ]:
#converting to word embeddings
'''
def get_word2vec_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # 300D vector

# Applying to dataset using the pretrained Word2Vec model
balanced_df["embedding"] = balanced_df["tokens"].apply(lambda x: get_word2vec_embedding(x, wv))'''
import gensim.downloader as api
import numpy as np
import pandas as pd

# Load the pretrained Word2Vec model
wv = api.load("word2vec-google-news-300")

# Function to get embeddings for each word in a review
def get_word_embeddings(tokens, model):
    return [model[word] for word in tokens if word in model]  # List of word embeddings

# Load dataset (assume it's preprocessed and tokenized)
#df = pd.read_pickle("amazon_reviews.pkl")  # Replace with actual dataset file

# Apply function to dataset
balanced_df["word_embeddings"] = balanced_df["tokens"].apply(lambda x: get_word_embeddings(x, wv))

###(b) Training Word2Vec model on our own datset

In [ ]:
# Training my custom Word2Vec model
custom_w2v = Word2Vec(
    sentences=balanced_df["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

# Example: Check words most similar to "excellent"
print(custom_w2v.wv.most_similar("excellent"))

In [ ]:
result_custom = custom_w2v.wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("Custom Word2Vec: King - Man + Woman =", result_custom[0][0])

In [ ]:
similarity_custom = custom_w2v.wv.similarity("excellent", "outstanding")
print("Custom Word2Vec similarity (excellent vs outstanding):", similarity_custom)

**Comparison based on King - Man + Woman on above 2 models :**

Google News Word2Vec : Returns "Queen"

Custom Word2Vec (Amazon Reviews) : Does not return "Queen" as the dataset may lacks historical context.


**Comparison based on Excellent vs Outstanding :**


Google News Word2Vec : High similarity (~0.9)


Custom Word2Vec (Amazon Reviews) :0.86168975 ( lower due to limited data.)


**Based on Context Understanding**

Google News Word2Vec : General knowledge	Biased towards Amazon review words.

Custom Word2Vec (Amazon Reviews): Biased towards Amazon review words.

**Conclusion:-**

Pretrained Word2Vec (Google News 300) performs better for general language tasks

It was trained on a large-scale dataset (Google News corpus).
It encodes rich semantic relationships between words.
It correctly captures King - Man + Woman = Queen.

Custom Word2Vec is more domain-specific (Amazon Reviews)

It is trained on customer reviews, so it understands sentiment-related words well.
But it lacks general world knowledge, making it weaker for analogy tasks.
Example: It may not return "Queen" for King - Man + Woman, but it might capture sentiment-related similarities better (e.g., "great" ∼ "awesome")

**Best Choice?**

If we need general word understanding, use then pretrained Word2Vec.

If we need domain-specific representations (Amazon reviews, legal, medical, etc.),  our own custom trained Word2V on reviews works better

In [ ]:
print(custom_w2v.wv.most_similar("excellent"))

### 3.

### Data cleaning and Data Preprocessing

In [ ]:
contractions_dict = {
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "how's": "how is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [ ]:
df=orig_df.copy()
df

In [ ]:
#from locale import D_FMT
import pandas as pd
import re
import nltk
from nltk.corpus import words
import re
#from spellchecker import SpellChecker
import pandas as pd
import re

# Initialize spell checker
#spell = SpellChecker()

# Function to filter misspelled words
def remove_misspelled_words(text):
    if isinstance(text, str):
        # Split text into words
        words = text.split()
        # Check each word and retain it only if it's correctly spelled
        corrected_words = [word for word in words if word in spell]
        return " ".join(corrected_words)
    return ""

# Download the English words corpus (only required once)
nltk.download('words')

# Get the set of valid English words
english_words = set(words.words())

# Function to filter non-English words from a string
def filter_non_english_words(text):
    if isinstance(text, str):
        # Split text into words, filter non-English words, and rejoin
        filtered_words = [word for word in text.split() if word.lower() in english_words]
        return " ".join(filtered_words)
    return ""

def expand_contractions(text, contractions_map):
    if isinstance(text, str):
        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions_map.keys()) + r')\b')
        return pattern.sub(lambda x: contractions_map[x.group()], text)
    return text

def preprocess_reviews(df):
    # Step 1: Convert all reviews to lowercase
    df['cleaned_review'] = df['review_body'].str.lower()

    # Step 2: Remove HTML tags and URLs
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else '')
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'http\S+', '', x) if isinstance(x, str) else '')

    # Step 3: Expand contractions (e.g., can't -> can not)
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: contractions.fix(x))
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: expand_contractions(x, contractions_dict))


    # Step 4: Separate concatenated words (e.g., slot...can't -> slot can't)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'(\w)(\'\w)', r'\1 \2', x))  # Insert space between word and contraction

    # Step 5: Replace multiple dots (two or more) with a space
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\.{2,}', ' ', x))

    # Step 6: Replace non-alphabetical characters (except for apostrophes and spaces) with a space
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^\w\s\'-]', ' ', x))  # Keep alphabet, spaces, apostrophes, and hyphens
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s\']', ' ', x))  # Keep only alphabet, spaces, and apostrophes
    # Step 7: Remove extra spaces (e.g., from replacing punctuation)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # Step 8:  Apply the function to remove misspelled words
    df['cleaned_review'] = df['cleaned_review'].apply(filter_non_english_words)
    #df['cleaned_review'] = df['cleaned_review'].apply(remove_misspelled_words)
    return df

def average_length(df, column='review_body'):
    # Replace non-string values with empty string
    df[column] = df[column].fillna('').astype(str)

    # Calculate average length
    lengths = df[column].apply(len)
    return lengths.mean()



# Calculate average lengths before preprocessing
avg_length_before = average_length(df, column='review_body')

# Preprocess the reviews
reduced_df = preprocess_reviews(df)

# Calculate average lengths after preprocessing
avg_length_after = average_length(df, column='cleaned_review')

# Output average lengths before and after
print(f"Average length before Data Cleaning: {avg_length_before}, Average length after Data Cleaning: {avg_length_after}")



#Average length before cleaning: 319.694505
#Average length after cleaning: 260.67447

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to get the WordNet POS tag from the NLTK POS tag
def get_wordnet_pos(tag):
    if tag.startswith('VB'):  # Verbs
        return wordnet.VERB
    elif tag.startswith('JJ'):  # Adjectives
        return wordnet.ADJ
    elif tag.startswith('NN'):  # Nouns
        return wordnet.NOUN
    elif tag.startswith('RB'):  # Adverbs
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    # Tokenize the sentence and POS tag each token
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)

    # Lemmatize using POS tags and filter out stopwords
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens if word.lower() not in stop_words
    ]

    # Return the lemmatized sentence as a string
    return ' '.join(lemmatized_words)

# Apply lemmatization to your dataframe column
df['cleaned_review'] = df['cleaned_review'].apply(lemmatize_text)

In [ ]:
print(f"Average length before data preprocessing: {avg_length_after}, Average length after data preprocessing: {average_length(reduced_df, column='cleaned_review')}")


In [ ]:
df_3classes = df.copy()
# Labels: Sentiment classification (Class 1 = Positive, Class 2 = Negative)
df = df[df["sentiment"].isin([1, 2])]  # Ignore Class 3 (Neutral)

In [ ]:
df_3classes.shape

In [ ]:
filtered_df = df[df['sentiment'] == 5]
print(filtered_df)

In [ ]:
### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))

In [ ]:
s="uiuiu hjhj jhj"
for word in s.split():
  print(word)

In [ ]:
### Tf-idf feature extraction
newdf= reduced_df
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit to top 5000 terms (optional)
    stop_words='english',  # Remove common stop words
    #ngram_range=(1,2)  # Use unigrams and bigrams
)
# Fit the vectorizer and transform the data
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_review'])

In [ ]:
tfidf_features

In [ ]:
# Train - Test Split
from sklearn.model_selection import train_test_split
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_tfidf, X_test_tfidf, _, _ = train_test_split(
    tfidf_features, df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

In [ ]:
print(type(X_train_google))  # <class 'list'>
print(type(X_train_google[0]))  # <class 'numpy.ndarray'>
print(X_train_google[0].shape)  # (300,)

In [ ]:
print(type(X_train_custom))  # <class 'list'>
print(type(X_train_custom[0]))  # <class 'numpy.ndarray'>
print(X_train_custom[0].shape)  # (300,)

In [ ]:
#Perceptron training
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, average="binary")
    train_recall = recall_score(y_train, y_train_pred, average="binary")
    train_f1 = f1_score(y_train, y_train_pred, average="binary")

    # Test metrics
    test_acc = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average="binary")
    test_recall = recall_score(y_test, y_test_pred, average="binary")
    test_f1 = f1_score(y_test, y_test_pred, average="binary")

    # Print results
    print(f"\n📌 {model_name} Results:")
    print(f"✅ Training Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")
    print(f"✅ Training Precision: {train_precision:.4f} | Test Precision: {test_precision:.4f}")
    print(f"✅ Training Recall: {train_recall:.4f} | Test Recall: {test_recall:.4f}")
    print(f"✅ Training F1 Score: {train_f1:.4f} | Test F1 Score: {test_f1:.4f}")

    #return {
    #    "train_acc": train_acc, "test_acc": test_acc,
    #    "train_precision": train_precision, "test_precision": test_precision,
    #    "train_recall": train_recall, "test_recall": test_recall,
    #    "train_f1": train_f1, "test_f1": test_f1
    #}

### **1️⃣ Perceptron Models**
print("\n🔹 Training Perceptron Models...")
perc_google = Perceptron()
perc_google.fit(X_train_google, y_train)
evaluate_model(perc_google, X_train_google, X_test_google, y_train, y_test, "Perceptron (Google News W2V)")

perc_custom = Perceptron()
perc_custom.fit(X_train_custom, y_train)
evaluate_model(perc_custom, X_train_custom, X_test_custom, y_train, y_test, "Perceptron (Custom W2V)")

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(perc_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "Perceptron (TF-IDF)")


### **2️⃣ SVM Models**
print("\n🔹 Training SVM Models...")
svm_google = LinearSVC(random_state=42, max_iter=10000)
svm_google.fit(X_train_google, y_train)
evaluate_model(svm_google, X_train_google, X_test_google, y_train, y_test, "SVM (Google News W2V)")

svm_custom = LinearSVC(random_state=42, max_iter=10000)
svm_custom.fit(X_train_custom, y_train)
evaluate_model(svm_custom, X_train_custom, X_test_custom, y_train, y_test, "SVM (Custom W2V)")

svm_tfidf =LinearSVC(random_state=42, max_iter=10000)
svm_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(svm_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "SVM (TF-IDF)")


### TF-IDF remains the best performing feature type for both Perceptron and SVM, achieving: Best test accuracy (84.1%) with SVM.

### SVM outperforms Perceptron consistently.

### Custom Word2Vec benefits from domain-specific training but still lags behind TF-IDF.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
df = df_3classes.copy()
df = df[df["sentiment"].isin([1, 2])]
from sklearn.model_selection import train_test_split

### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))



In [ ]:
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary :
df_3classes["embedding_google"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)


In [ ]:
y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))

In [ ]:
y_train_ternary = np.array(y_train_ter .replace({1: 0, 2: 1,3:2}))
y_test_ternary = np.array(y_test_ter.replace({1: 0, 2: 1,3:2}))

In [ ]:
#creating a Feedforward MLP with two hidden layers (50 & 10 nodes).
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)  # First hidden layer (50 neurons)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)  # Second hidden layer (10 neurons)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(10, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [ ]:
def train_model(X_train, y_train, X_test, y_test, num_classes, epochs=250, lr=0.001):
    # Convert numpy arrays to PyTorch tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

    # Define model
    model = MLP(input_size=300, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluate on test set
    with torch.no_grad():
        test_outputs = model(X_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = accuracy_score(y_test.numpy(), predicted.numpy())

    return accuracy


In [ ]:
# Training Binary Classification Models:
print("Binary Classification (Positive vs Negative)")

# Train on Google Word2Vec
acc_binary_google = train_model(X_train_google, y_train_binary, X_test_google, y_test_binary, num_classes=2)
print()
print(f"✅ Google W2V Accuracy (Binary): {acc_binary_google:.4f}")
print()
# Train on Custom Word2Vec
acc_binary_custom = train_model(X_train_custom, y_train_binary, X_test_custom, y_test_binary, num_classes=2)
print(f"✅ Custom W2V Accuracy (Binary): {acc_binary_custom:.4f}")




In [ ]:
print("Ternary Classification (Positive, Negative, Neutral)")
#X_train_google_ternary, X_test_google_ternary
#X_train_custom_ternary, X_test_custom_ternary
# Train on Google Word2Vec
acc_ternary_google = train_model(X_train_google_ternary, y_train_ternary, X_test_google_ternary, y_test_ternary, num_classes=3)
print(f"✅ Google W2V Accuracy (Ternary): {acc_ternary_google:.4f}")

# Train on Custom Word2Vec
acc_ternary_custom = train_model(X_train_custom_ternary, y_train_ternary, X_test_custom_ternary, y_test_ternary, num_classes=3)
print(f"✅ Custom W2V Accuracy (Ternary): {acc_ternary_custom:.4f}")


###(b)

In [ ]:
def concat_w2v_vectors(tokens, model, num_vectors=10):
    # Use model[word] directly for KeyedVectors (Google W2V) and model.wv[word] for Word2Vec models
    vectors = [model[word] if isinstance(model, gensim.models.KeyedVectors) else model.wv[word] for word in tokens if word in model]

    # If fewer than num_vectors words exist, pad with zero vectors
    while len(vectors) < num_vectors:
        vectors.append(np.zeros(model.vector_size))

    # Take only the first num_vectors
    vectors = vectors[:num_vectors]

    # Flatten (concatenate) the vectors
    return np.concatenate(vectors, axis=0)

# Convert dataset to concatenated vectors
X_google_concat = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df["cleaned_review"]])
X_custom_concat = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df["cleaned_review"]])



In [ ]:
print("Training Data Shape:", X_google_concat.shape)

In [ ]:
#Similarly for Ternary , get the 3000 input vector size:
# Convert dataset to concatenated vectors
X_google_concat_tern = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df_3classes["cleaned_review"]])
X_custom_concat_tern = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df_3classes["cleaned_review"]])

In [ ]:
print("Ternary training Data Shape:", X_google_concat_tern.shape)

In [ ]:
X_google_concat = X_google_concat.astype(np.float32)
X_custom_concat = X_custom_concat.astype(np.float32)
X_google_concat_tern = X_google_concat_tern.astype(np.float32)
X_custom_concat_tern = X_custom_concat_tern.astype(np.float32)


In [ ]:
from sklearn.model_selection import train_test_split

# Binary classification split (one at a time)
X_train_google_concat_binary, X_test_google_concat_binary, y_train_google_concat_binary, y_test_google_concat_binary = train_test_split(
    X_google_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

del X_google_concat  # Free memory

X_train_custom_concat_binary, X_test_custom_concat_binary, y_train_custom_concat_binary, y_test_custom_concat_binary = train_test_split(
    X_custom_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

del X_custom_concat  # Free memory

# Ternary classification split
X_train_google_concat_ternary, X_test_google_concat_ternary, y_train_google_concat_ternary, y_test_google_concat_ternary = train_test_split(
    X_google_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])

del X_google_concat_tern  # Free memory



In [ ]:
X_train_custom_concat_ternary, X_test_custom_concat_ternary, y_train_custom_concat_ternary, y_test_custom_concat_ternary = train_test_split(
    X_custom_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])

del X_custom_concat_tern  # Free memory


In [ ]:
'''
#Splitting binary into train and test : 80-20
from sklearn.model_selection import train_test_split
# Binary
X_train_google_concat_binary, X_test_google_concat_binary, y_train_google_concat_binary, y_test_google_concat_binary = train_test_split(
    X_google_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

X_train_custom_concat_binary, X_test_custom_concat_binary, y_train_custom_concat_binary, y_test_custom_concat_binary = train_test_split(
    X_custom_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#Ternary

X_train_google_concat_ternary, X_test_google_concat_ternary, y_train_google_concat_ternary, y_test_google_concat_ternary = train_test_split(
    X_google_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])

X_train_custom_concat_ternary, X_test_custom_concat_ternary, y_train_custom_concat_ternary, y_test_custom_concat_ternary = train_test_split(
    X_custom_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])
'''

In [ ]:
'''X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary :
df_3classes["embedding_google"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)'''